In [1]:
import json
import os
from shapely.geometry import Polygon

# 환경에 맞춰서 folder_path 수정
folder_path = 'data/kor_data/Training/label/my/' # 전처리할 데이터 폴더 위치
img_path = 'data/kor_data/Training/img/my/' # 원본 이미지 폴더 위치
output_file_path = 'data/kor_data/Training/pre_data.json' # 출력 경로

file_list = os.listdir(folder_path) # 폴더에 있는 파일 이름을 리스트로 불러옴
# print(file_list) #출력 : 00110011002.json

# img_list = os.listdir(img_path)
img_list = [os.path.splitext(file)[0] + '.jpg' for file in file_list if file.endswith(".json")]
# print(img_list)

img_path_li = []
for i in img_list:
    img_path_li.append(f'{img_path}{i}')

file_path_li = [] # 파일 경로와 리스트 한 번에 넣을 리스트 생성
#경로+파일명을 리스트에 저장
for i in file_list:
    file_path_li.append(f'{folder_path}{i}')
# print(file_path_li) # 출력 : ../mmocr/data/kor_data/label/001/00110011002.json ...



In [2]:
my_dict = {}
num_instance = 0

for item in file_path_li:
    with open(item, 'r', encoding='utf-8') as fp:
        data = json.load(fp) # 파일을 파이썬의 딕셔너리 형태로 변환
    
    if 'data_list' not in my_dict:
        my_dict['metainfo'] = {"dataset_type": "TextDetDataset",
                            "task_name": "textdet",
                            "category":[{"id": 0, "name": "text"}]}
        
    if 'data_list' not in my_dict:
        my_dict['data_list'] = []
        # print(data['data_list'])
        
    my_dict['data_list'].append({'instances':[],
                            'img_path': f'{img_path_li[num_instance]}',
                            'height': data['image']['height'],
                            'width': data['image']['width']}
                            )

    # print(len(data['text']['word']))
    
    for i in range(len(data['text']['word'])):
        my_dict['data_list'][num_instance]['instances'].append({'polygon': [],
                                                                'bbox': [],
                                                                'bbox_label': 0,
                                                                'ignore': 'false'})

        # 폴리건 형태로 변형해줄 거 만들기
        xmin, ymin, xmax, ymax = data['text']['word'][i]['wordbox']
        polygon_coords = [xmin, ymin, xmax, ymin, xmax, ymax, xmin, ymax]
        # shapely_polygon = Polygon(polygon_coords)
        
        my_dict['data_list'][num_instance]['instances'][i]['polygon'] = polygon_coords
        my_dict['data_list'][num_instance]['instances'][i]['bbox'] = data['text']['word'][i]['wordbox']

    num_instance += 1

with open(output_file_path, 'w') as json_file:
    json.dump(my_dict, json_file)

In [ ]:
# print(data['text']['word'][1]['wordbox'])
# print(data['text']['word'][1]['wordbox'][0])
# print(data['text']['word'][1]['wordbox'][1])
# print(data['text']['word'][1]['wordbox'][2])
# print(data['text']['word'][1]['wordbox'][3])
# xmin, ymin, xmax, ymax = data['text']['word'][1]['wordbox']
# print(xmin, ymin, xmax, ymax)